# Imports

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns
import google.colab
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import scipy.stats as stats
from sklearn.feature_selection import f_classif

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
music_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/Music/cleaned_df.csv")

In [29]:
music_df

,artist,song,emotion,variance,genre,release date,key,tempo,loudness,explicit,...,speechiness,liveness,acousticness,instrumentalness,speechiness_log,instrumentalness_bin,mode_int,key_root,cos_pitch,sin_pitch
0,ABBA,She's My Kind Of Girl,joy,0.447619,pop,2014,F Maj,128,-6.00,No,...,3,31,7,0,1.386294,0,1,5,-8.660254e-01,5.000000e-01
1,ABBA,"Andante, Andante",love,0.202222,pop,1980,A# Maj,102,-10.72,No,...,2,7,68,0,1.098612,0,1,10,5.000000e-01,-8.660254e-01
2,ABBA,As Good As New,sadness,0.300881,pop,1979,E Maj,139,-5.70,No,...,3,8,20,2,1.386294,1,1,4,-5.000000e-01,8.660254e-01
3,ABBA,Bang,joy,0.355000,pop,1975,F Maj,132,-3.00,No,...,3,32,3,0,1.386294,0,1,5,-8.660254e-01,5.000000e-01
4,ABBA,Bang-A-Boomerang,joy,0.355000,pop,1975,F Maj,132,-3.00,No,...,3,32,3,0,1.386294,0,1,5,-8.660254e-01,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236964,(Eri Sasaki),Gate Of Steiner,sadness,0.833514,other,2018,D min,148,-6.44,No,...,4,14,20,0,1.609438,0,0,2,5.000000e-01,8.660254e-01
236965,(Elena Tsagrinou),El Diablo,love,0.833514,other,2021,A min,114,-7.78,No,...,13,84,0,0,2.639057,0,0,9,-1.836970e-16,-1.000000e+00
236966,(Dima Bilan),Believe,joy,0.833514,other,2009,C min,134,-6.72,No,...,5,22,5,0,1.791759,0,0,0,1.000000e+00,0.000000e+00
236967,(Ani Lorak),Shady Lady,joy,0.833514,other,2009,F# min,128,-13.00,No,...,5,7,5,0,1.791759,0,0,6,-1.000000e+00,1.224647e-16


In [33]:
import pandas as pd
import numpy as np
import random

# Load your dataset
df = music_df.copy()

# Define core feature columns for similarity
feature_columns = [
    'tempo', 'loudness', 'energy', 'danceability', 'positiveness',
    'speechiness', 'liveness', 'acousticness', 'instrumentalness',
    'cos_pitch', 'sin_pitch'
]

# Build emotion index
emotion_index = {emo: df[df['emotion'] == emo] for emo in df['emotion'].unique()}

# Mock: define favorite artists
mock_favorite_artists = random.sample(df['artist'].unique().tolist(), 5)
print(f"✅ Mock favorite artists: {mock_favorite_artists}")

# Mock: build similar artist list (we'll just pick random similar ones)
artist_sim_cache = {}
for artist in mock_favorite_artists:
    similar = random.sample(df['artist'].unique().tolist(), 5)
    artist_sim_cache[artist] = similar

# Mock: create a random user taste vector (same dimensionality as features)
user_vector = np.random.rand(len(feature_columns))

# Recommendation function
def recommend_songs(current_mood, top_n=10):
    print(f"\n🎯 Generating recommendations for mood '{current_mood}'")

    # Step 1: Filter songs by mood
    mood_df = emotion_index.get(current_mood)
    if mood_df is None or mood_df.empty:
        print("⚠️ No songs found for this mood.")
        return []

    # Step 2: Filter by favorite + similar artists
    preferred_artists = set(mock_favorite_artists)
    for artist in mock_favorite_artists:
        preferred_artists.update(artist_sim_cache.get(artist, []))

    filtered_df = mood_df[mood_df['artist'].isin(preferred_artists)]
    if filtered_df.empty:
        print("⚠️ No songs matched favorite artists, using all mood songs.")
        filtered_df = mood_df

    # Step 3: Compute similarity
    feature_matrix = filtered_df[feature_columns].values
    cand_norms = np.linalg.norm(feature_matrix, axis=1, keepdims=True)
    cand_normed = feature_matrix / cand_norms
    user_normed = user_vector / np.linalg.norm(user_vector)
    scores = cand_normed.dot(user_normed)

    # Step 4: Get top N
    top_idx = np.argpartition(scores, -top_n)[-top_n:]
    top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]
    top_songs = filtered_df.iloc[top_idx]

    print("\n🎵 Recommended Songs:")
    for rank, row in top_songs.iterrows():
        print(f"{row['artist']} — {row['song']} (emotion: {row['emotion']})")

    return top_songs

# Example usage
recommend_songs("joy", top_n=5)


✅ Mock favorite artists: ['Einstrzende Neubauten', 'Charles Aznavour', 'Westside Gunn', 'Hot Milk', 'Phone-EG']

🎯 Generating recommendations for mood 'joy'

🎵 Recommended Songs:
Herb Alpert — Spanish Flea (emotion: joy)
Westside Gunn — Sensational Sherri (emotion: joy)
Quality Control, Quavo & Nicki Minaj — She For Keeps (emotion: joy)
Remy Ma — Conceited (emotion: joy)
MC Serch — Back to the Grill (emotion: joy)


,artist,song,emotion,variance,genre,release date,key,tempo,loudness,explicit,...,speechiness,liveness,acousticness,instrumentalness,speechiness_log,instrumentalness_bin,mode_int,key_root,cos_pitch,sin_pitch
172928,Herb Alpert,Spanish Flea,joy,0.833514,jazz,1959,A# Maj,81,-10.00,No,...,7,7,48,88,2.079442,1,1,10,0.500000,-0.866025
67460,Westside Gunn,Sensational Sherri,joy,0.833514,rap,2019,C Maj,78,-5.88,Yes,...,32,28,20,0,3.496508,0,1,0,1.000000,0.000000
112107,"Quality Control, Quavo & Nicki Minaj",She For Keeps,joy,0.833514,other,2017,G# min,130,-5.44,Yes,...,12,12,2,0,2.564949,0,0,8,-0.500000,-0.866025
108496,Remy Ma,Conceited,joy,0.833514,rap,2006,B Maj,100,-7.81,Yes,...,45,16,20,0,3.828641,0,1,11,0.866025,-0.500000
138814,MC Serch,Back to the Grill,joy,0.833514,rap,1992,B Maj,101,-10.00,Yes,...,25,11,11,0,3.258097,0,1,11,0.866025,-0.500000


In [34]:
import pandas as pd
import numpy as np
import random

# Load your dataset


# Prepare core features
feature_columns = [
    'tempo', 'loudness', 'energy', 'danceability', 'positiveness',
    'speechiness', 'liveness', 'acousticness', 'instrumentalness',
    'cos_pitch', 'sin_pitch'
]
emotions = df['emotion'].unique()
users = [f"user_{i}" for i in range(1, 101)]

# Synthetic history list
history = []

# Generate synthetic sessions
for _ in range(5000):
    user = random.choice(users)
    song = df.sample(1).iloc[0]
    mood_before = random.choice(emotions)
    song_emotion = song['emotion']

    # Simulate mood change (basic rules)
    if mood_before in ["sadness", "anger"]:
        mood_after = "joy" if song_emotion in ["joy", "love"] else mood_before
    else:
        mood_after = mood_before if song_emotion in ["sadness", "anger"] else "joy"

    mood_improved = int(mood_before != mood_after)

    record = {
        "user": user,
        "song_id": song['song'],
        "artist": song['artist'],
        "mood_before": mood_before,
        "mood_after": mood_after,
        "mood_improved": mood_improved
    }
    for feat in feature_columns:
        record[feat] = song[feat]

    history.append(record)

# Create DataFrame
history_df = pd.DataFrame(history)
print("✅ Synthetic listening history created!")
history_df.head()


✅ Synthetic listening history created!


,user,song_id,artist,mood_before,mood_after,mood_improved,tempo,loudness,energy,danceability,positiveness,speechiness,liveness,acousticness,instrumentalness,cos_pitch,sin_pitch
0,user_55,When The Stone Begins To Turn,Jackson Browne,fear,joy,1,154,-14.60,33,70,70,16,4,2,0,-8.660254e-01,-5.000000e-01
1,user_73,Toni,2 Chainz,fear,fear,0,100,-5.71,60,84,40,8,10,11,0,-8.660254e-01,-5.000000e-01
2,user_78,Combination Pizza Hut and Taco Bell,Das Racist,fear,joy,1,130,-7.00,90,80,89,22,18,39,0,-1.000000e+00,1.224647e-16
3,user_7,In Good Hands,Navy Blue,sadness,sadness,0,79,-14.00,62,38,47,22,44,1,0,-5.000000e-01,8.660254e-01
4,user_13,Came Back Haunted,Nine Inch Nails,fear,joy,1,130,-6.24,65,69,55,5,14,0,6,-1.836970e-16,-1.000000e+00


In [ ]:
import pandas as pd
import numpy as np
import random
from collections import Counter

# Load your dataset
df = music_df.copy()

# Define feature columns
feature_columns = [
    'tempo', 'loudness', 'energy', 'danceability', 'positiveness',
    'speechiness', 'liveness', 'acousticness', 'instrumentalness',
    'cos_pitch', 'sin_pitch'
]
emotions = df['emotion'].unique()
users = [f"user_{i}" for i in range(1, 101)]

# Generate synthetic history
history = []
for _ in range(5000):
    user = random.choice(users)
    song = df.sample(1).iloc[0]
    mood_before = random.choice(emotions)
    song_emotion = song['emotion']
    mood_after = "joy" if mood_before in ["sadness", "anger"] and song_emotion in ["joy", "love"] else mood_before
    mood_improved = int(mood_before != mood_after)
    record = {
        "user": user,
        "song_id": song['song'],
        "artist": song['artist'],
        "mood_before": mood_before,
        "mood_after": mood_after,
        "mood_improved": mood_improved
    }
    for feat in feature_columns:
        record[feat] = song[feat]
    history.append(record)

history_df = pd.DataFrame(history)
print("✅ Synthetic listening history created!")

# Encode features for ML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

X_raw = history_df[feature_columns + ['mood_before']]
y = history_df['mood_improved']

encoder = OneHotEncoder(sparse_output=False)
mood_encoded = encoder.fit_transform(X_raw[['mood_before']])
mood_encoded_df = pd.DataFrame(mood_encoded, columns=encoder.get_feature_names_out(['mood_before']))

X_full = pd.concat([X_raw[feature_columns].reset_index(drop=True), mood_encoded_df], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_full)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
print("✅ Random Forest model trained!")

# Prepare emotion index (for fast filtering)
emotion_index = {emo: df[df['emotion'] == emo] for emo in df['emotion'].unique()}

# Build similar artists mock cache
artist_sim_cache = {}
for artist in df['artist'].unique():
    similar = random.sample(df['artist'].unique().tolist(), 5)
    artist_sim_cache[artist] = similar

# Helper: get top favorite artists for a user
def get_user_favorite_artists(history_df, user_id, top_n=5):
    user_history = history_df[history_df['user'] == user_id]
    artist_counts = Counter(user_history['artist'])
    top_artists = [artist for artist, _ in artist_counts.most_common(top_n)]
    return top_artists

# Recommender function (with ML scoring)
def recommend_songs(user_id, current_mood, top_n=10):
    print(f"\n🎯 Recommendations for {user_id} in mood '{current_mood}'")

    # Step 1: Get user favorite + similar artists
    favorite_artists = get_user_favorite_artists(history_df, user_id)
    preferred_artists = set(favorite_artists)
    for artist in favorite_artists:
        preferred_artists.update(artist_sim_cache.get(artist, []))

    # Step 2: Filter songs by mood + artist
    mood_df = emotion_index.get(current_mood)
    if mood_df is None or mood_df.empty:
        print("⚠️ No songs found for this mood.")
        return []

    filtered_df = mood_df[mood_df['artist'].isin(preferred_artists)]
    if filtered_df.empty:
        print("⚠️ No artist match, using all mood songs.")
        filtered_df = mood_df

    # Step 3: Prepare features for ML model
    X_cand = filtered_df[feature_columns].copy()
    mood_onehot = encoder.transform([[current_mood]] * len(filtered_df))
    X_cand_full = np.hstack([X_cand.values, mood_onehot])
    X_cand_scaled = scaler.transform(X_cand_full)

    # Step 4: Predict mood improvement probability
    probs = rf.predict_proba(X_cand_scaled)[:, 1]

    # Step 5: Select top N
    top_idx = np.argsort(probs)[-top_n:][::-1]
    top_songs = filtered_df.iloc[top_idx]

    print("\n🎵 Top Recommendations (highest predicted mood improvement):")
    for _, row in top_songs.iterrows():
        print(f"{row['artist']} — {row['song']} (emotion: {row['emotion']})")

    return top_songs

# Example run
recommend_songs("user_1", "joy", top_n=5)


✅ Synthetic listening history created!
✅ Random Forest model trained!
